In [2]:
from tomoSegmentPipeline.utils.common import read_array, write_array
from tomoSegmentPipeline.utils import setup

from cryoS2Sdrop.dataloader import singleCET_dataset, randomRotation3D
from cryoS2Sdrop.model import Denoising_3DUNet
from cryoS2Sdrop.losses import self2self_L2Loss
from cryoS2Sdrop.trainer import denoisingTrainer

import os
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from torchsummary import summary
from pytorch_lightning import Trainer
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import LearningRateMonitor

PARENT_PATH = setup.PARENT_PATH

%matplotlib inline
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [2]:
# cet_path = os.path.join(PARENT_PATH, 'data/raw_cryo-ET/tomo02.mrc') 
# cet_path = os.path.join(PARENT_PATH, 'data/S2SDenoising/dummy_tomograms/tomo02_dummy.mrc')
cet_path = 
p=0.3 # dropout probability
subtomo_length = 128
n_features = 48
tensorboard_logdir = os.path.join(PARENT_PATH, 'data/S2SDenoising/tryout_model_logs')
batch_size = 4
epochs = 2
num_gpus = 2
lr = 1e-4

transform = randomRotation3D(0.5)

s2s_trainer = denoisingTrainer(cet_path, subtomo_length, lr, n_features, p, tensorboard_logdir)

In [3]:
s2s_trainer.train(batch_size, epochs, num_gpus, accelerator = 'gpu', strategy = 'dp', transform=transform)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Size of dataset: 32, Steps per epoch: 4. 



LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

   | Name      | Type             | Params
------------------------------------------------
0  | loss_fn   | self2self_L2Loss | 0     
1  | EB1       | PartialConv3d    | 1.3 K 
2  | EB2       | Sequential       | 62.3 K
3  | EB3       | Sequential       | 62.3 K
4  | EB4       | Sequential       | 62.3 K
5  | EB5       | Sequential       | 62.3 K
6  | EB6       | Sequential       | 62.3 K
7  | EB_bottom | Sequential       | 62.3 K
8  | up65      | Upsample         | 0     
9  | up54      | Upsample         | 0     
10 | up43      | Upsample         | 0     
11 | up32      | Upsample         | 0     
12 | up21      | Upsample         | 0     
13 | DB5       | Sequential       | 497 K 
14 | DB4       | Sequential       | 622 K 
15 | DB3       | Sequential       | 622 K 
16 | DB2       | Sequential       | 622 K 
17 | DB1       | Sequential       | 223 K 
------------------------------------------------
3.0 M     Trainable params
0         Non

# Writing additional hyperparams to hparams.yaml

In [5]:
import yaml

In [8]:
hparams_file = os.path.join(s2s_trainer.tensorboard_logdir, 'version_%i' %s2s_trainer.model.logger.version)
hparams_file = os.path.join(hparams_file, 'hparams.yaml')
hparams_file

'/home/ubuntu/Thesis/data/S2SDenoising/tryout_model_logs/version_30/hparams.yaml'

In [11]:
extra_hparams = {'transform':transform, 'singleCET_dataset.Vmask_probability':0.1}
# sdump = yaml.dump(extra_hparams)

# with open(hparams_file, "a") as fo:
#     fo.write(sdump)

# Loading model

In [10]:
ckpt_file = os.path.join(PARENT_PATH, 'data/S2SDenoising/tryout_model_logs/version_0/checkpoints/epoch=1-step=126.ckpt')

model = Denoising_UNet.load_from_checkpoint(ckpt_file)